<h1 align="center">Spatial Data with Python Workshop</h1>

<h4 align="center"><i>Created by <a href="https://github.com/amitparikh1">Amit Parikh</a> for Carolina Analytics and Data Science on 3/1/2023</i></h4>

 
**Spatial Data** can be defined as any series of data that has an implicit or explicit association with a geographic location. Spatial data is all around us and is constantly being collected. 

In this workshop we will go over some common examples of spatial datasets, data types that are specific to spatial data, how to best analyze spatial data in Python using GeoPandas, and explore ways we can visualize spatial data. 

This workshop is meant to be **introductory** and is open to all skill levels. A basic understanding of Python and Pandas would be helpful, but not necessary. To familiarize yourself with Python and Pandas, check out our <a href="https://youtube.com/playlist?list=PLtOYSqZWWG7L1tUaEgUwtJP0d9xYHN7o1"> CADS intro YouTube series </a>. 

## Examples of Spatial Data

Try to think of some examples of spatial data, it's all around us! <a href="https://www.safegraph.com/guides/uses-of-geospatial-data">Here </a> are some use cases for spatial data analysis.

## Data Types Specific to Spatial Data <a href="https://www.safe.com/what-is/spatial-data/">(Source)</a>

### Vector

Vector data is the most common type of spatial data. Data in this form comes in three types: **point**, **lines**, and **polygons**.

<img src="https://datacarpentry.org/organization-geospatial/fig/dc-spatial-vector/pnt_line_poly.png" width =250/>

**Points** are represented by a **(latitude, longitude)** coordinate pair that corresponds to a physical location in the world. 

    Example: the precise location of a thermometer recording daily temperatures.

**Lines** are represented by **2+** points that are **connected together**. Each "bend" in a line is represented by a new point. 

    Example: a line can be used to mark a GPS route from point A to point B. 
    
**Polygons** are represented by **3+** points that are **connected together** and **closed**. 

    Example: a polygon can mark the boundary of a county, zipcode, state, or country. 
<img src="https://digital.newberry.org/ahcb/images/statepages/North_Carolina.gif" width =250/>

### Raster

Raster data is gridded data where each pixel is associated with a specific geographical location.

    Example: commonly used to represent satellite image data. 
    
<img src="https://datacarpentry.org/organization-geospatial/fig/dc-spatial-raster/raster_concept.png" width =250/>

### Attributes

Attributes refer to additional data that describes a geographical feature. 

    Example: A polygon that represents a home has attributes such as # of bedrooms, # of bathrooms, square footage etc. 

### Geocoding

The process by which tabular data such as address is analyzed and converted into latitude, longitude coordinates. 

    Example: Say a Real Estate investor had the street addresses of 100 homes in his portfolio! To visualize where these homes are on a map, you would first geocode the addresses to lat, lng coordinates. 
    
Many API services offer free tiers for geocoding such as <a href="https://developers.google.com/maps">Google Maps </a> and <a href= "https://developer.here.com/"> HERE </a>.

## Using GeoPandas to Analyze Spatial Data

GeoPandas is a library that extends Pandas to work more nicely with spatial data. 

### Get GeoPandas setup

In [ ]:
# Installations
# If you don't have geopandas installed, uncomment this line
# !pip install geopandas 
# !pip install mapclassify

In [ ]:
# Imports
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

### Read in the data

In [ ]:
# Use a provided sample dataset given by GeoPandas
df_world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
df_world.head(10)

### Take a look at the data

In [ ]:
# What are the data types in each column
df_world.info()

In [ ]:
# How is the geometry column stored
print(type(df_world['geometry']))

The geometry column has a special **geometry** data type and is stored as a **geoseries**.

The 3 basic classes of geometric objects are:

    - Points / Multi-Points
    - Lines / Multi-Lines
    - Polygons / Multi-Polygons

Geoseries have special attributes that may be useful in spatial analysis:

    * Area
    * Centroid 
    * geom_type
and so much more...

Let's look at **centroid** for example.

In [ ]:
# Centroid (returns a Point that is at the center of the geometry)
print(df_world['geometry'].centroid)

In [ ]:
# Note: Centroid of a Multi-Polygon may not be inside any of the sub-polygons
indonesia = df_world[df_world['name'] == 'Indonesia']
base_indonesia = indonesia.plot(figsize=(10,10))
indonesia.centroid.plot(ax=base_indonesia, color='r', figsize=(10,10))
plt.title("Map of Indonesia and its centroid")
plt.show()

### Aggregation with dissolve <a href="https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html">(GeoPandas Docs)</a>

Right now we can plot Population (or other metrics) on the country level

In [ ]:
df_world.plot('pop_est', figsize=(10,10))
plt.title("Population by Country")
plt.show()

What if we want to look at the data on the continent level?

In [ ]:
# 8 continents is much less granular and more manageable
num_continents = df_world['continent'].nunique()
print(f'We have {num_continents} different continents.')

# Dissolve allows us to aggregate geometric features (for non-geometric features it performs a Pandas GroupBy)
df_continent = df_world.dissolve('continent', aggfunc='sum').reset_index()
df_continent

In [ ]:
# Plot the new DataFrame
df_continent.plot('pop_est', figsize=(10,10))
plt.title("Population by Continent")
plt.show()

This concept can be applied to geographies of all granularities. For example aggregating and disolving:

    * county level to state level
    * state level to country level
    
**any smaller geography can be aggregated up to a larger geographical level**, but not the other way around. 

### Spatial Joins

Spatial joins consist of two types. 

**Binary predicate joins** (uses the function *GeoDataFrame.sjoin()*):
    
    * intersect
    * contains 
    * within
    * touches
    * crosses 
    * overlaps
    
and **nearest joins** (uses the function *GeoDataFrame.sjoin_nearest()*): based on proximity between the geometries being merged.

Let's practice spatial joins by joining cities to countries based on their geometry. If a city is within a country, we will match them up! This will use a **binary predicate join**. 

In [ ]:
# Read in a new data frame for cities that we will use for joining
df_cities = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
df_cities.head()

In [ ]:
# Focus on the countries and geometries in df_world
df_countries = df_world[['name', 'geometry']]
df_countries = df_countries.rename(columns={'name': 'country'})

In [ ]:
df_cities_and_countries = df_cities.sjoin(df_countries, predicate = 'within')
df_cities_and_countries.sample(10)

Notice the index_right column. We won't dive into this but I recommend reading about spatial indexes <a href="https://learn.microsoft.com/en-us/sql/relational-databases/spatial/spatial-indexes-overview?view=sql-server-ver16"> here </a>.

### Interactive Mapping

Geopandas can make interactive maps based on the folium library. 

In [ ]:
# The simplest option using the GeoDataFrame.explore() function
df_world.explore()

In [ ]:
# Let's add some customization
df_world.explore(
     column="continent", # make choropleth based on "BoroName" column
     tooltip="name", # show "BoroName" value in tooltip (on hover)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap="Set1", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black"), # use black outline
    )

In [ ]:
# Adding multiple tiles
cont = df_world.explore(
     column="continent", # make choropleth based on "BoroName" column
     tooltip="name", # show "BoroName" value in tooltip (on hover)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap="Set1", # use "Set1" matplotlib colormap
     style_kwds=dict(color="black"), # use black outline
     name="countries",
     max_bounds=True,
    )

df_cities.explore(
     m=cont, # pass the map object
     color="red", # use red color on all points
     tooltip="name", # show "name" column in the tooltip
     tooltip_kwds=dict(labels=False), # do not show column label in the tooltip
     name="cities", # name of the layer in the map
     max_bounds=True,
)

folium.TileLayer('Stamen Toner', control=True).add_to(cont)  # use folium to add alternative tiles
folium.LayerControl().add_to(cont)  # use folium to add layer control

cont  # show map

Try playing around with this on your own time and merging in other interesting spatial data.

## Thanks for coming and any questions?